## LangChain
LangChain is a framework for developing applications powered by language models.

- GitHub: 

https://github.com/hwchase17/langchain

- Docs: 

https://python.langchain.com/en/latest/


- Features:
        Interface LLMs
        Prompt Templates
        Chains (Construct sequences of calls)
        Agents and Tools
        Memory (Persist application state between runs of a chain)
        Document Loaders
        Indexer

In [1]:
# !pip install langchain
# !pip install openai
# !pip install huggingface_hub
# !pip install wikipedia
# !pip install sentence_transformers
# !pip install faiss-cpu

In [2]:
import os
import json
import pandas as pd
import numpy as np
import requests
import langchain

In [3]:
# https://platform.openai.com/account/api-keys
os.environ['OPENAI_API_KEY'] = ''

# https://huggingface.co/settings/tokens
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ''

In [4]:
dir(langchain)

['Anthropic',
 'ArxivAPIWrapper',
 'Banana',
 'BaseCache',
 'BasePromptTemplate',
 'CerebriumAI',
 'Cohere',
 'ConversationChain',
 'ElasticVectorSearch',
 'FAISS',
 'FewShotPromptTemplate',
 'ForefrontAI',
 'GoogleSearchAPIWrapper',
 'GoogleSerperAPIWrapper',
 'GooseAI',
 'HuggingFaceHub',
 'HuggingFacePipeline',
 'HuggingFaceTextGenInference',
 'InMemoryDocstore',
 'LLMBashChain',
 'LLMChain',
 'LLMCheckerChain',
 'LLMMathChain',
 'LlamaCpp',
 'MRKLChain',
 'Modal',
 'OpenAI',
 'Optional',
 'PALChain',
 'Petals',
 'PipelineAI',
 'PowerBIDataset',
 'Prompt',
 'PromptTemplate',
 'QAWithSourcesChain',
 'ReActChain',
 'SQLDatabase',
 'SQLDatabaseChain',
 'SagemakerEndpoint',
 'SearxSearchWrapper',
 'SelfAskWithSearchChain',
 'SerpAPIChain',
 'SerpAPIWrapper',
 'StochasticAI',
 'VectorDBQA',
 'VectorDBQAWithSourcesChain',
 'Wikipedia',
 'WikipediaAPIWrapper',
 'WolframAlphaAPIWrapper',
 'Writer',
 '__all__',
 '__annotations__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__

## 1. LLMs
A generic interface for all LLMs. 

In [5]:
from langchain.llms import OpenAI

In [6]:
llm = OpenAI()

text = "What would be a good company name for a company that provides AI based recommendation and search engines?"
res = llm(text)

print(res.strip())

SmartMatch Solutions


In [7]:
from langchain import HuggingFaceHub

In [8]:
llm = HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":0, "max_length":64})
llm("translate English to German: How old are you?")

## 2. Prompt Templates

LangChain faciliates prompt management and optimization.

Normally when you use an LLM in an application, you are not sending user input directly to the LLM. Instead, you need to take the user input and construct a prompt, and only then send that to the LLM.

In [15]:
from langchain import PromptTemplate

In [17]:
llm("Can Barack Obama have a conversation with George Washington?")

'\n\nNo, it is not possible for Barack Obama to have a conversation with George Washington, as George Washington passed away in 1799, more than 200 years before Barack Obama was born.'

In [18]:
template = """
Question: {question}

Let's think step by step.

Answer:
"""

In [19]:
prompt = PromptTemplate(template=template, input_variables=["question"])
prompt.format(question="Can Mahatma Gandhi have a conversation with Sachin Tendulkar?")

"\nQuestion: Can Mahatma Gandhi have a conversation with Sachin Tendulkar?\n\nLet's think step by step.\n\nAnswer:\n"

In [20]:
llm(prompt)  # TypeError: Object of type PromptTemplate is not JSON serializable

ValueError: Argument `prompt` is expected to be a string. Instead found <class 'langchain.prompts.prompt.PromptTemplate'>. If you want to run the LLM on multiple prompts, use `generate` instead.

## 3. Chains
Combine LLMs and Prompts in multi-step workflowsm

In [21]:
from langchain import LLMChain

In [22]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Can Barack Obama have a conversation with George Washington?"
print(llm_chain.run(question))

No, Barack Obama cannot have a conversation with George Washington. George Washington died in 1799, so it would be impossible to have a conversation with him.


In [25]:
# If there are multiple variables, you can input them all at once using a dictionary.

prompt = PromptTemplate(
    template="What is a good name for {company} that makes {product}?",
    input_variables=["company", "product"],
)

chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({
    'company': "ABC Startup",
    'product': "colorful socks"
    }).strip())

Colorful Socks Co.


## 4. Agents and Tools
Agents involve an LLM making decisions about which Actions to take, taking that Action, seeing an Observation, and repeating that until done.

When used correctly agents can be extremely powerful. In order to load agents, you should understand the following concepts:

    Tool: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains.
    LLM: The language model powering the agent.
    Agent: The agent to use.


In [5]:
from langchain.agents import load_tools , initialize_agent
from langchain.llms import OpenAI     

In [6]:
llm = OpenAI(temperature=0, model='text-davinci-001')

In [7]:
tools = load_tools(["wikipedia", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [9]:
agent.run("In what year was the film 3 Idiots released? What is this release year raised to the 2 power?")

## 5. Memory
Add State to Chains and Agents.

Memory is the concept of persisting state between calls of a chain/agent. LangChain provides a standard interface for memory, a collection of memory implementations, and examples of chains/agents that use memory.

In [44]:
from langchain import OpenAI, ConversationChain
llm = OpenAI(temperature=0)

In [46]:
conversation = ConversationChain(llm=llm, verbose=True)
conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

In [47]:
conversation.predict(input="My name is Gautam. What do you think about Indian Premiere League?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: My name is Gautam. What do you think about Indian Premiere League?
AI:

> Finished chain.


" Hi Gautam! It's nice to meet you. I think the Indian Premier League is a great way to bring cricket fans together and create a sense of community. It's also a great way to promote the sport and bring in new fans."

In [48]:
conversation.predict(input="""Which team do you think has the most loyal fanbase in this league ?
It is known to win hearts everytime and has qualified many times in playoffs bt never won any trophy""")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: My name is Gautam. What do you think about Indian Premiere League?
AI:  Hi Gautam! It's nice to meet you. I think the Indian Premier League is a great way to bring cricket fans together and create a sense of community. It's also a great way to promote the sport and bring in new fans.
Human: Which team do you think has the most loyal fanbase in this league ?
It is known to win hearts everytime and has qualified many times in playoffs bt never won any trophy
AI:

> Finished chain.


" That's a tough question! I don't have enough information to answer that definitively, but I do know that the Chennai Super Kings have a very loyal fanbase. They have been in the playoffs many times and have come close to winning the trophy, but have never been able to take it home."

## 6. Document Loaders
Combining language models with your own text data is a powerful way to differentiate them. The first step in doing this is to load the data into “documents” - a fancy way of say some pieces of text. This module is aimed at making this easy.


In [10]:
from langchain.document_loaders import NotionDirectoryLoader , TextLoader
loader = NotionDirectoryLoader("Notion_DB")

In [12]:
docs = loader.load()
docs

[]

## 7. Indexes
Indexes refer to ways to structure documents so that LLMs can best interact with them. This module contains utility functions for working with documents

- Embeddings: An embedding is a numerical representation of a piece of information, for example, text, documents, images, audio, etc.
- Text Splitters: When you want to deal with long pieces of text, it is necessary to split up that text into chunks.
- Vectorstores: Vector databases store and index vector embeddings from NLP models to understand the meaning and context of strings of text, sentences, and whole documents for more accurate and relevant search results.

In [48]:
url = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/extras/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
    f.write(res.text)

    
# Document Loader
loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()
documents[0].page_content[:1050]

'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with 

In [49]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [50]:
docs[0]

Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.', metadata={'source':

In [52]:
# Embeddings

from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-mpnet-base-v2')

text = "This is a test document."
query_result = embeddings.embed_query(text)
doc_result = embeddings.embed_documents([text])

In [55]:
len(query_result)

768

In [56]:
# Vectorstore: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html

from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

query = "Where did the scandal happened"
res = db.similarity_search(query)
print(res[0].page_content)

We can’t change how divided we’ve been. But we can change how we move forward—on COVID-19 and other issues we must face together. 

I recently visited the New York City Police Department days after the funerals of Officer Wilbert Mora and his partner, Officer Jason Rivera. 

They were responding to a 9-1-1 call when a man shot and killed them with a stolen gun. 

Officer Mora was 27 years old. 

Officer Rivera was 22. 

Both Dominican Americans who’d grown up on the same streets they later chose to patrol as police officers. 

I spoke with their families and told them that we are forever in debt for their sacrifice, and we will carry on their mission to restore the trust and safety every community deserves. 

I’ve worked on these issues a long time. 

I know what works: Investing in crime preventionand community police officers who’ll walk the beat, who’ll know the neighborhood, and who can restore trust and safety.


In [57]:
db.save_local("faiss_index")

In [58]:
new_db = FAISS.load_local("faiss_index", embeddings)
res = new_db.similarity_search(query)
print(res[0].page_content)

We can’t change how divided we’ve been. But we can change how we move forward—on COVID-19 and other issues we must face together. 

I recently visited the New York City Police Department days after the funerals of Officer Wilbert Mora and his partner, Officer Jason Rivera. 

They were responding to a 9-1-1 call when a man shot and killed them with a stolen gun. 

Officer Mora was 27 years old. 

Officer Rivera was 22. 

Both Dominican Americans who’d grown up on the same streets they later chose to patrol as police officers. 

I spoke with their families and told them that we are forever in debt for their sacrifice, and we will carry on their mission to restore the trust and safety every community deserves. 

I’ve worked on these issues a long time. 

I know what works: Investing in crime preventionand community police officers who’ll walk the beat, who’ll know the neighborhood, and who can restore trust and safety.
